In [22]:
from FFNN import NeuralNetwork
from activation_functions import ReLU, leaky_ReLU, sigmoid, mse
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


### Generate data
Using the same function and seed for generating data as in project 1.

In [23]:
def generate_data(n, noise = True):
    # Fixed seed value to ensure consisten results across runs
    seed = 42
    np.random.seed(seed)
    # Creating an array with equally spaced data
    x = np.linspace(-1, 1, n)
    # Runges equation with noise
    if noise:
        y = 1 / (1 + 25*x**2) +  np.random.normal(0, 0.1, size=n)
    else:
        y = 1 / (1 + 25*x**2)
    return x, y


In [24]:
x, y = generate_data(50)
x = x.reshape(-1,1)
y = y.reshape(-1,1)

In [25]:

# 2️⃣ Split into training and test sets
X_train_full, X_test, y_train_full, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

# 3️⃣ Create a validation set from the training data
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.25, random_state=42
)
# (→ 60% train, 20% val, 20% test)



In [26]:
# 4️⃣ Scale features
scaler = StandardScaler()
x_train = scaler.fit_transform(X_train)
x_val   = scaler.transform(X_val)
x_test  = scaler.transform(X_test)



In [29]:
# 5️⃣ Initialize the neural network
nn = NeuralNetwork(
    network_input_size=X_train.shape[1],
    layer_output_sizes=[50, 100, 1],
    activation_funcs=[ReLU, sigmoid],
    cost_func=mse)

# 6️⃣ Train using your chosen optimizer
nn.train_network(nn, X_train, y_train, eta=0.01, epochs=100)

# 7️⃣ Monitor validation performance
y_val_pred = nn.predict(X_val)
val_loss = mse(y_val, y_val_pred)
print("Validation loss:", val_loss)

# 8️⃣ Evaluate on the test set (only once, after final tuning)
y_test_pred = nn.predict(X_test)
test_loss = mse(y_test, y_test_pred)
print("Final test loss:", test_loss)

NameError: name 'layer' is not defined